In [1]:
import gspread
from google.oauth2.service_account import Credentials
import re

In [2]:
# Setup auth and client
SCOPES = ['https://www.googleapis.com/auth/drive']
creds = Credentials.from_service_account_file('credentials.json', scopes=SCOPES)
client = gspread.authorize(creds)

In [3]:
# Open your spreadsheet by name or URL
spreadsheet = client.open("AGIMBA Contacts 25-26")

In [4]:
# Read emails from two sheets/ranges
sheet_agimba_contacts = spreadsheet.worksheet("AGIMBA Contacts")
sheet_agmb = spreadsheet.worksheet("AGMBRoster")
sheet_email_lists = spreadsheet.worksheet("EmailLists")  # The sheet to write into

In [5]:
def split_emails(raw_list):
    split_result = []
    for entry in raw_list:
        # Split on anything that isn't a valid email character
        parts = re.split(r'[^a-zA-Z0-9._%+\-@]+', entry)
        split_result.extend([p.strip() for p in parts if p.strip()])
    return split_result

In [6]:
# Basic email pattern: username@domain
EMAIL_REGEX = re.compile(r"^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$")
def is_valid_email(email):
    return EMAIL_REGEX.match(email) is not None

In [7]:
# Read columns from both sheets
emails_agimba_contacts = sheet_agimba_contacts.col_values(3)[1:]  # 'AGIMBA Contacts' column C
emails_agmb = sheet_agmb.col_values(6)[1:]  # 'AGMBRoster' column F

In [8]:
# Combine, split multi-email entries, remove blanks
emails_combined = split_emails( emails_agimba_contacts + emails_agmb )

In [9]:
# Remove duplicates (case-insensitive), preserve first casing
seen = {email.lower() for email in emails_combined if is_valid_email(email) }
update_list = [ [email,] for email in sorted(seen) ]

In [10]:
# Clear old data
sheet_email_lists.batch_clear(['A2:A'])

# Prepare for writing
sheet_email_lists.update(range_name=f'A2:A{len(seen)+1}', values=update_list )

print(f"Wrote {len(update_list)} unique emails to '{sheet_email_lists.title}' sheet.")

Wrote 87 unique emails to 'EmailLists' sheet.
